---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [8]:
def graph_identification():
    
    # Your Code Here
    from itertools import groupby
    
    algos = []
    for g in P1_Graphs:
        # import matplotlib.pyplot as plt
        # plt.figure()
        # plt.hist(list(g.degree().values()))
        # plt.show()
        graph_degrees = sorted(g.degree().values())
        degrees = [len(list(group)) for (degree, group) in groupby(graph_degrees)]
        shortest_path_length = nx.average_shortest_path_length(g)
        clustering = nx.average_clustering(g)
        if len(degrees)>10:
            algos.append('PA')
        elif clustering>0.1:
            algos.append('SW_L')
        else:
            algos.append('SW_H')
        
    return algos # Your Answer Here
                  

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [2]:
G = nx.read_gpickle('email_prediction.txt')

#print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [24]:
def salary_predictions():
    
    # Your Code Here
    
    # from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.preprocessing import MinMaxScaler
    
    df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
    df['node_degree'] = pd.Series(G.degree())
    df['node_clustering'] = pd.Series(nx.clustering(G))
    # centrality measures
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness_centrality'] = pd.Series(nx.closeness_centrality(G))
    df['betweenness_centrality'] = pd.Series(nx.betweenness_centrality(G))
    # link analysis
    hub, auth = nx.hits(G)
    df['hub_score'] = pd.Series(hub)
    df['auth_score'] = pd.Series(auth)
    df['pagerank'] = pd.Series(nx.pagerank(G))
    
    knowns_df = df[df['ManagementSalary'].notnull()]
    unknowns_df = df[df['ManagementSalary'].isnull()]
    
    X_train = knowns_df.loc[:, knowns_df.columns!='ManagementSalary']
    y_train = knowns_df['ManagementSalary']
    X_test = unknowns_df.loc[:, unknowns_df.columns!='ManagementSalary']
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = GradientBoostingClassifier(random_state=0)
    
    clf.fit(X_train_scaled, y_train)
    y_predict_prob = clf.predict_proba(X_test_scaled)[:, 1]
    ans = pd.Series(y_predict_prob, index=X_test.index)
    
    return ans # X_train.shape, X_test.shape   # Your Answer Here

#salary_predictions()

1       0.014725
2       0.984922
5       0.988773
8       0.037200
14      0.043611
18      0.015994
27      0.033635
30      0.925348
31      0.179832
34      0.020842
37      0.014672
40      0.016120
45      0.021468
54      0.447861
55      0.798434
60      0.082457
62      0.988764
65      0.984220
77      0.029112
79      0.017208
97      0.008553
101     0.003304
103     0.373655
108     0.016380
113     0.043663
122     0.001749
141     0.091140
142     0.988764
144     0.009921
145     0.897872
          ...   
913     0.007544
914     0.007081
915     0.001388
918     0.031972
923     0.011458
926     0.035735
931     0.007592
934     0.001132
939     0.001051
944     0.001136
945     0.011458
947     0.197480
950     0.019120
951     0.003310
953     0.003704
959     0.001054
962     0.001051
963     0.220448
968     0.018145
969     0.030189
974     0.024848
984     0.001136
987     0.026717
989     0.019538
991     0.021016
992     0.001051
994     0.000978
996     0.0009

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [22]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [48]:
def new_connections_predictions():
    
    # Your Code Here
    
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.preprocessing import MinMaxScaler
    
    pref = nx.preferential_attachment(G)
    df_pref = pd.DataFrame.from_dict({(f,t):v for (f,t,v) in pref}, orient='index')
    df_pref.columns = ['pref_attachment']
    
    ra = nx.resource_allocation_index(G)
    df_ra = pd.DataFrame.from_dict({(f,t):v for (f,t,v) in ra}, orient='index')
    df_ra.columns = ['res_alloc']
    
    jc = nx.jaccard_coefficient(G)
    df_jc = pd.DataFrame.from_dict({(f,t):v for (f,t,v) in jc}, orient='index')
    df_jc.columns = ['jaccard']
    
    aa = nx.adamic_adar_index(G)
    df_aa = pd.DataFrame.from_dict({(f,t):v for (f,t,v) in aa}, orient='index')
    df_aa.columns = ['adamic_adar']
    
    sh = nx.cn_soundarajan_hopcroft(G, community='Department')
    df_sh = pd.DataFrame.from_dict({(f,t):v for (f,t,v) in sh}, orient='index')
    df_sh.columns = ['soundarajan_hopcraft']
    
    ra_sh = nx.ra_index_soundarajan_hopcroft(G, community='Department')
    df_ra_sh = pd.DataFrame.from_dict({(f,t):v for (f,t,v) in ra_sh}, orient='index')
    df_ra_sh.columns = ['ra_soundarajan_hopcraft']
    
    knowns_df = future_connections[future_connections['Future Connection'].notnull()]
    unknowns_df = future_connections[future_connections['Future Connection'].isnull()]
    
    final_df = df_pref.join(df_ra).join(df_jc).join(df_aa).join(df_sh).join(df_ra_sh).join(future_connections)
    
    knowns_df = final_df[final_df['Future Connection'].notnull()]
    unknowns_df = final_df[final_df['Future Connection'].isnull()]
    
    X_train = knowns_df.loc[:, knowns_df.columns!='Future Connection']
    y_train = knowns_df['Future Connection']
    X_test = unknowns_df.loc[:, unknowns_df.columns!='Future Connection']
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = GradientBoostingClassifier(random_state=0)
    
    clf.fit(X_train_scaled, y_train)
    y_predict_prob = clf.predict_proba(X_test_scaled)[:, 1]
    ans = pd.Series(y_predict_prob, index=X_test.index)
    
    return ans # Your Answer Here

#new_connections_predictions()

(0, 9)          0.027223
(0, 19)         0.056372
(0, 20)         0.213043
(0, 35)         0.018397
(0, 38)         0.016055
(0, 42)         0.855937
(0, 43)         0.014873
(0, 50)         0.013649
(0, 53)         0.567935
(0, 54)         0.037775
(0, 62)         0.341194
(0, 63)         0.055392
(0, 69)         0.027226
(0, 72)         0.012976
(0, 83)         0.178004
(0, 90)         0.025634
(0, 91)         0.017795
(0, 95)         0.106358
(0, 100)        0.027223
(0, 106)        0.987540
(0, 108)        0.018395
(0, 109)        0.012976
(0, 110)        0.012976
(0, 118)        0.015531
(0, 122)        0.025634
(0, 131)        0.037396
(0, 133)        0.728237
(0, 140)        0.059529
(0, 149)        0.124862
(0, 151)        0.016055
                  ...   
(988, 989)      0.012976
(988, 996)      0.012976
(988, 997)      0.035198
(988, 1000)     0.012976
(988, 1002)     0.012976
(989, 994)      0.012976
(989, 996)      0.012976
(989, 1003)     0.012976
(989, 1004)     0.012976
